### This is the script to develop 2045 household control totals from REMI population forecast
Need three input data tables:    
1, REMI population forecast (total pop by year, race, gender, age and large area)    
2, Synthesized persons table (with large area)  
3, Synthesized households table   
4, REMI HH pop and GQ pop distribution by age group

In [51]:
import pandas as pd
from numpy import *
import itertools

import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)  #DEBUG, INFO, WARNING, ERROR, CRITICAL

# set up inputs

In [52]:
# folders & files
inputdir = './inputs/'
outputdir = './outputs/'
syn_hh_file = 'syn_households.csv'  # synthesized HH file
syn_hhpop_file = 'syn_persons.csv'  # synthesized (hh pop)persons file
hh_gq_file='REMIHHPOPCONTROLS_ByYear2015_20145_TotalsbyLargeArea.xlsx' #hhpop and group quarter distribution by age group in each large area

years = range(2015, 2046)
baseyear = 2015

# final household attributes, including large area, to be used in control totals
control_attrs = ['large_area_id', 'race_id', 'age_of_head', 'persons', 'children', 'cars', 'workers', 'income']

# define bins for digitization
bin_census_agegrp =[0, 5, 10, 15, 18, 20, 21, 22, 25, 30, 35, 40, 45, 50, 55, 60, 61, 65, 66, 70, 75, 80, 85]
ctrl_bins = {'age_of_head':[0, 5, 18, 25, 35, 65], 
             'persons':[1,2,3,4,5,6,7,8,9,10], 
             'children':[0,1,2,3], 
             'cars':[0,1,2,3], 
             'workers':[0,1,2,3]
            }




## Step 1. Compute REMI household population
#### using the ratios between base year REMI pop and Census HH pop to produce REMI HH pop for all forecast years. #

In [53]:
# read input excel files and combine

dic_la = {"detroit": 5, "rest of wayne": 3, "macomb": 99, "livingston": 93,
          "monroe": 115, "oakland": 125, "st clair": 147, "washtenaw": 161}
dic_race = {"white nh": 1, "black nh": 2, "hispanic": 3, "other nh": 4}
dic_gend = {"male": 1, "female": 2}

remi_total_pop = pd.DataFrame()

for la in dic_la.keys():
    print 'processing table ' + "control " + la + ".xlsx", 
    for sheet in itertools.product(dic_gend.keys(), dic_race.keys()):
        print '.', 
        dfp = pd.read_excel(inputdir + "control " + la + ".xlsx", sheetname=" ".join(sheet), header=0)
        dfp['large_area_id'] = dic_la[la]
        dfp['gender'] = dic_gend[sheet[0]]
        dfp['race_id'] = dic_race[sheet[1]]
        remi_total_pop = remi_total_pop.append(dfp.round())
    print 'done'
    
remi_total_pop.drop(range(1990, 2015), axis=1, inplace=True)
remi_total_pop.index.name = 'age_group'
remi_total_pop.reset_index(inplace=True)
remi_total_pop['age_group'] = remi_total_pop['age_group'].str.replace('Age.|\+|" "', '').astype(int)
logging.debug(remi_total_pop.head())

#remi_total_pop.sum(axis=0)
#remi_total_pop.groupby('large_area_id').sum().to_csv('remi_pop_total.csv')

processing table control washtenaw.xlsx . . . . . . . . done
processing table control rest of wayne.xlsx . . . . . . . . done
processing table control livingston.xlsx . . . . . . . . done
processing table control monroe.xlsx . . . . . . . . done
processing table control oakland.xlsx . . . . . . . . done
processing table control st clair.xlsx . . . . . . . . done
processing table control detroit.xlsx . . . . . . . . done
processing table control macomb.xlsx . . . . . . . . done


In [54]:
# 1.2 recode age to to GQ agegrp 
remi_total_pop['age_group'] = digitize(remi_total_pop['age_group'], ctrl_bins['age_of_head'])
logging.debug(remi_total_pop.age_group.unique())

In [55]:
## use remi total and HH pop to get GQ pop
#REMI Total POP by large area
remi_total_pop_la = remi_total_pop.groupby('large_area_id').sum()[years]
logging.debug(remi_total_pop_la.head(2))

#REMI HH POP by large area
remi_hhpop_la = pd.read_excel(inputdir + hh_gq_file, sheetname="REMIHHPOPULATION", index_col=[0], header=0)
logging.debug(remi_hhpop_la.head(2))
#remi_hhpop_la.sum()

#GQ pop by large area
gq_pop_la = remi_total_pop_la-remi_hhpop_la
logging.debug(gq_pop_la.head(2))

In [56]:
# read GQ proportion by age group
gq_portion = pd.read_excel(inputdir + hh_gq_file, sheetname="GQratios", header=0)
gq_portion['age_group']=digitize(gq_portion['age_group'],ctrl_bins['age_of_head'])
gq_portion.set_index(['large_area_id'], inplace=True)
logging.debug(gq_portion.head())

In [57]:
#calculate GQ pop for age groups and LA
gq_pop_la_agp = pd.merge(gq_portion, gq_pop_la, left_index=True, right_index=True, how='left')
gq_pop_la_agp.set_index('age_group', append=True, inplace=True)
gq_pop_la_agp[years] = gq_pop_la_agp[years].multiply(gq_pop_la_agp.gq_portion, axis='index').round(0)
gq_pop_la_agp.to_csv("gq_pop_forecast.csv")
logging.debug(gq_pop_la_agp.head(2))

In [8]:
#calculate REMI HH pop by age group and LA
remi_hhpop_la_agp = remi_total_pop.groupby(['large_area_id','age_group'])[years].sum() - gq_pop_la_agp
remi_hhpop_la_agp.drop('gq_portion', axis =1, inplace=True)
logging.debug(remi_hhpop_la_agp.head())

In [9]:

# 1.3 Aggregate total population by large area, gender, race and age groups
remi_total_pop_lgra = remi_total_pop.groupby(['large_area_id', 'gender', 'race_id', 'age_group']).sum()
remi_total_pop_lgra.reset_index(inplace=True)
logging.debug(remi_total_pop_lgra.head(2))

In [10]:
#one step, calculate cell percentage as sum of la and age group 
remi_total_pop_lgra[years]= \
        remi_total_pop_lgra.groupby(['large_area_id','age_group'])[years].apply(lambda x: x.astype(float)/x.sum())
lagr_ratios = remi_total_pop_lgra.set_index(['large_area_id','age_group', 'gender', 'race_id'])    
logging.debug(lagr_ratios.head(2))

In [11]:
hhpops_lagr_sum = pd.merge(remi_total_pop_lgra[['large_area_id','age_group', 'gender', 'race_id']], 
         remi_hhpop_la_agp, left_on=['large_area_id','age_group'], right_index=True, how='left')
hhpops_lagr_sum.set_index(['large_area_id','age_group', 'gender', 'race_id'], inplace=True)
logging.debug(hhpops_lagr_sum.head(2))

In [12]:
remi_hhpop_lagr=hhpops_lagr_sum.multiply(lagr_ratios).round()
logging.debug(remi_hhpop_lagr.head(3))

## Step 2. Convert REMI HH population to REMI households

In [13]:
#1.4 read and aggregate synthesized(census) population data (aggregated by lgra)
syn_hhpop = pd.read_csv(inputdir + syn_hhpop_file)
syn_hhpop['age_group'] = digitize(syn_hhpop['age'], ctrl_bins['age_of_head'])
syn_hhpop.rename(columns = {'sex':'gender'},inplace=True)
syn_hhpop_lagr = syn_hhpop.groupby(['large_area_id', 'gender', 'race_id', 'age_group']).size().to_frame()
syn_hhpop_lagr.columns=['hh_pop']
logging.debug(syn_hhpop_lagr.sum()-remi_hhpop_lagr.sum()[2015])

In [14]:
remi_hhpop_lagr.reset_index(inplace=True)
remi_hhpop_lagr.rename(columns={'age_group': 'age_of_head'}, inplace=True)

# aggregate REMI HH pop by large area, race and age groups
remi_hhpop_lar = remi_hhpop_lagr.groupby(['large_area_id', 'race_id', 'age_of_head'])[years].sum()
logging.debug(remi_hhpop_lar.head(2)) 

In [15]:
###Compute synthesized HHs by 3 attributes 'large_area_id','race','age_of_head'
#read synthesized HHs
syn_hhs=pd.read_csv(inputdir + syn_hh_file,sep=',',header=0)

# recode 'age_of_head', 'persons', 'workers','cars','children'
for attr in ['age_of_head','persons','workers','cars','children']:
    syn_hhs[attr] = digitize(syn_hhs[attr], ctrl_bins[attr])
    
    
# process 'income' into quartile and get bins
syn_hhs.loc[syn_hhs['income']<0,'income' ]=0
syn_hhs['income'], inc_bins=pd.qcut(syn_hhs['income'], 4, labels=False, retbins=True)
syn_hhs['income']= syn_hhs['income'] + 1
ctrl_bins['income']=list(inc_bins)
logging.debug(ctrl_bins['income'])
    
#aggregate synthesized HHs3 by 3 attributes 'large_area','race','age_of_head'
syn_hhs_lar=syn_hhs.groupby(['large_area_id','race_id','age_of_head']).size().to_frame()
syn_hhs_lar.columns=['HHs_lar']

logging.debug(syn_hhs_lar.head(2)) 
logging.debug(syn_hhs[pd.isnull(syn_hhs).any(axis=1)]) 

In [16]:
if syn_hhs_lar.shape[0]<>remi_hhpop_lar.shape[0]:
    print "  * Warning, Syn HHs and REMI HH pop have different sub-categories", syn_hhs_lar.shape[0],remi_hhpop_lar.shape[0]

  * Warning, Syn HHs and REMI HH pop have different sub-categories 135 192


In [17]:
remi_hhpop_lagr.reset_index(inplace=True)
remi_hhpop_la = remi_hhpop_lagr.groupby('large_area_id').sum()
remi_hhpop_la.drop(['gender','race_id','age_of_head'],axis=1, inplace=True)
remi_hhpop_la.to_csv('remi_hhpop_bylarge.csv')
logging.debug(remi_hhpop_la)

In [18]:
# join synthesized HH and census HH pop and calculate forecast HHs
hh_hhpop_join=pd.merge(syn_hhs_lar, remi_hhpop_lar,left_index=True,right_index=True,how='outer')

In [19]:
hh_hhpop_join.columns

Index([u'HHs_lar',       2015,       2016,       2017,       2018,       2019,
             2020,       2021,       2022,       2023,       2024,       2025,
             2026,       2027,       2028,       2029,       2030,       2031,
             2032,       2033,       2034,       2035,       2036,       2037,
             2038,       2039,       2040,       2041,       2042,       2043,
             2044,       2045],
      dtype='object')

In [20]:
#calculate ratio between baseline HHs and HH pop
hh_hhpop_join.fillna(0, inplace=True)
hh_hhpop_join.ratio = hh_hhpop_join['HHs_lar']/hh_hhpop_join[baseyear]

#apply ratio to forecast years to get future households by lra


In [21]:
remi_hhs = hh_hhpop_join.multiply(hh_hhpop_join.ratio, axis='index')
logging.debug(remi_hhs.head())

In [22]:
remi_hhs.head()

HHs_lar      2015           2016  \
large_area_id race_id age_of_head                                           
3             1       1                 0.000000       0.0       0.000000   
                      2                 0.033739      65.0      64.297196   
                      3               647.328989    6634.0    6445.675467   
                      4             17052.843946   42305.0   42813.703204   
                      5            123496.582816  207014.0  205775.538321   

                                            2017           2018  \
large_area_id race_id age_of_head                                 
3             1       1                 0.000000       0.000000   
                      2                63.504596      62.504891   
                      3              6406.156589    6441.967523   
                      4             42688.341321   42282.426799   
                      5            204129.028575  202221.224891   

                                            2019           2020  \
large_area_id race_id age_of_head                                 
3             1       1                 0.000000       0.000000   
                      2                61.590831      60.686114   
                      3              6504.124377    6435.917720   
                      4             41794.281331   41464.954455   
                      5            199891.055664  197656.932838   

                                            2021           2022  \
large_area_id race_id age_of_head                                 
3             1       1                 0.000000       0.000000   
                      2                59.876904      59.161642   
                      3              6324.776972    6224.369747   
                      4             41183.998723   40786.146059   
                      5            195477.097109  193154.086619   

                                            2023      ...        \
large_area_id race_id age_of_head                     ...         
3             1       1                 0.000000      ...         
                      2                58.523721      ...         
                      3              6154.504273      ...         
                      4             40236.730474      ...         
                      5            190941.440008      ...         

                                            2036           2037  \
large_area_id race_id age_of_head                                 
3             1       1                 0.000000       0.000000   
                      2                56.604247      56.522755   
                      3              5549.816641    5537.717034   
                      4             37991.503178   37893.954703   
                      5            178295.529284  178553.840415   

                                            2038           2039  \
large_area_id race_id age_of_head                                 
3             1       1                 0.000000       0.000000   
                      2                56.440744      56.362366   
                      3              5549.914219    5569.820024   
                      4             37747.631990   37602.115463   
                      5            179157.560655  180001.098619   

                                            2040           2041  \
large_area_id race_id age_of_head                                 
3             1       1                 0.000000       0.000000   
                      2                56.282950      56.209244   
                      3              5591.579802    5605.630959   
                      4             37438.459757   37392.104077   
                      5            180880.430273  181776.465650   

                                            2042           2043  \
large_area_id race_id age_of_head                                 
3             1       1                 0.000000       0.000000   
          

## Step 3. Extend REMI HHs attributes 

In [23]:
#aggregate synthesized HHs by all HH attributes
syn_hhs_sum_all=syn_hhs.groupby(control_attrs).size().to_frame()
syn_hhs_sum_all.columns = ['HHs_all']

#reindex synthetic HHs by large, race and age
syn_hhs_sum_all_lar=syn_hhs_sum_all.reset_index().set_index(['large_area_id','race_id','age_of_head'])

logging.debug(syn_hhs_sum_all_lar.head())

In [24]:
#merge synthetic HHs aggregated by all attributes to synthetic HHs aggregated by lra
syn_hhs_join=pd.merge(syn_hhs_sum_all_lar,syn_hhs_lar, left_index=True, right_index=True, how='outer')
syn_hhs_join['ratio'] = syn_hhs_join['HHs_all']/syn_hhs_join['HHs_lar']
logging.debug(syn_hhs_join.head(3))

In [25]:
#merge syn hhs with ratio to REMI hhs
remi_hhs_ratio=pd.merge(syn_hhs_join,remi_hhs, left_index=True, right_index=True, how='outer')
remi_hhs_ratio.reset_index(inplace=True)
remi_hhs_ratio.set_index(control_attrs,inplace=True)
logging.debug(remi_hhs_ratio.head(3))

In [26]:
#remi_hhs_83ratio.ratio = remi_hhs_83ratio['HHs_all']/remi_hhs_83ratio['HHs3']
remi_hhs_ratio=remi_hhs_ratio.multiply(remi_hhs_ratio.ratio, axis='index')
remi_hhs_ratio.fillna(0, inplace=True)
#remi_hhs_83ratio.head()

#drop unwanted columns
remi_hhs_ratio=remi_hhs_ratio[years]

logging.debug(remi_hhs_ratio.head(3))

In [27]:
#filter out rows with 0s from 2010 to 2040, this could reduce table size significantly
hhs_filtered=remi_hhs_ratio[remi_hhs_ratio.sum(axis=1)>0]
logging.debug(hhs_filtered.head())

### Step 4: estimate total persons through HH size, the scale the HHs to meet REMI HH pop target

In [28]:
hhs_base = hhs_filtered.copy()
hhs_base.reset_index(inplace=True)

In [29]:
hhs_size_la = hhs_base.groupby(['large_area_id','persons'])[years].sum().reset_index()
hhs_size_la.loc[hhs_size_la['persons']==10.0, 'persons']=11.1  #change 10+ hh size to 11.1 (average from transition result)

In [30]:
hhs_size_la[years]=hhs_size_la[years].multiply(hhs_size_la.persons,axis=0)

In [31]:
ctr_persons_la = hhs_size_la.groupby('large_area_id')[years].sum() # estimated hh pop by large area by year

In [32]:
hhs_base.set_index('large_area_id',inplace=True)
hhs_base[years]=hhs_base[years]*remi_hhpop_la[years]/ctr_persons_la[years]

In [33]:
#verify the adjustment, estimated persons by year should be slightly less than REMI HH pop by year
logging.debug(hhs_base[years].multiply(hhs_base.persons,axis=0).sum(axis=0)-remi_hhpop_la.sum(axis=0))

In [34]:
hhs_base.reset_index(inplace=True)
hhs_base.set_index(control_attrs,inplace=True)
hhs_base=hhs_base.round(0)
logging.debug(hhs_base.head())

## Step 5. format control total table

In [35]:

#stack household control table and add year to index
dfhh=pd.DataFrame(hhs_base.stack(),columns=['total_number_of_households'])
indn=dfhh.index.names[:-1]+['year']
dfhh.index.names=indn
logging.debug(dfhh.head(5))

In [36]:
#round 'total_number_of_households'
dfcc=dfhh.reset_index()

dfcc['total_number_of_households']=pd.Series.round(dfcc['total_number_of_households'],0)
logging.debug(dfcc.head())

In [37]:
#review output
print unique(dfcc['income'])
dft=dfcc[dfcc.year.isin([2015,2020,2025,2030,2035,2040,2045])]
logging.debug(dft.groupby(['year','income'])['total_number_of_households'].sum())

[ 1.  2.  3.  4.]


In [38]:
for hhattr in ctrl_bins.keys():
    fmt_bin = ctrl_bins[hhattr] + [0]
    colmin, colmax=hhattr+'_min',hhattr+'_max'
    dfcc[colmin] = dfcc[hhattr].apply(lambda x: fmt_bin[int(x)-1])
    dfcc[colmax] = dfcc[hhattr].apply(lambda x: fmt_bin[int(x)]) -1


In [39]:
for hhattr in ctrl_bins.keys():   
    print dfcc.groupby([hhattr+'_min',hhattr + '_max'])['total_number_of_households'].sum()

cars_min  cars_max
0          0           5450205.0
1          1          22877410.0
2          2          22781626.0
3         -1          10651684.0
Name: total_number_of_households, dtype: float64
workers_min  workers_max
0             0             20923189.0
1             1             23014606.0
2             2             14397321.0
3            -1              3425809.0
Name: total_number_of_households, dtype: float64
persons_min  persons_max
1             1             18742804.0
2             2             20477042.0
3             3              9187585.0
4             4              7824878.0
5             5              3462412.0
6             6              1334402.0
7             7               450300.0
8             8               167541.0
9             9                55520.0
10           -1                58441.0
Name: total_number_of_households, dtype: float64
income_min  income_max 
0.000       26248.210      15773037.0
26249.210   54726.794      15910943.0
54727.

In [48]:
writer = pd.ExcelWriter('output.xlsx')
dft=dfcc[dfcc.year.isin([2015,2020,2025,2030,2035,2040,2045])]
for col in ctrl_bins.keys():
    ss = dft.groupby(['year',col])['total_number_of_households'].sum()
    ss.to_frame().to_excel(writer,col)


In [50]:
ss.to_frame()

total_number_of_households
year children                            
2015 1.0                        1298046.0
     2.0                         238418.0
     3.0                         209009.0
     4.0                         124697.0
2020 1.0                        1340720.0
     2.0                         236630.0
     3.0                         206815.0
     4.0                         123516.0
2025 1.0                        1397144.0
     2.0                         233702.0
     3.0                         202869.0
     4.0                         121033.0
2030 1.0                        1449901.0
     2.0                         232880.0
     3.0                         200814.0
     4.0                         119909.0
2035 1.0                        1487086.0
     2.0                         234337.0
     3.0                         201613.0
     4.0                         120686.0
2040 1.0                        1506209.0
     2.0                         237313.0
     3.0                         204169.0
     4.0                         122336.0
2045 1.0                        1510097.0
     2.0                         240817.0
     3.0                         207263.0
     4.0                         124384.0

In [40]:
removefields=control_attrs[:]
removefields.remove('large_area_id')
removefields.remove('race_id')

dfcc.drop(removefields,1,inplace=True)
dfcc=dfcc.astype(int64)
dfcc=dfcc.set_index('year')


logging.debug(dfcc.head(5))

In [41]:
dfcc.to_csv("annual_household_control_totals_gqadj.csv")

In [42]:
#households growth 1.13731894257 (before scale by HH pop)
#persons growth 1.06802522683 (before scale by HH pop)


#2045/2015 HHs growth
print 'households growth', dfcc.loc[2045].sum()['total_number_of_households'].astype(float)/dfcc.loc[2015].sum()['total_number_of_households']
#2045/2015 persons growth
print 'persons growth', remi_hhpop_lar[2045].sum()/remi_hhpop_lar[2015].sum()

households growth 1.11356775053
persons growth 1.07807516133


In [43]:
dfcc.loc[2045].sum()['total_number_of_households'].astype(float)/ dfcc.loc[2015].sum()['total_number_of_households']

1.1135677505253534

### End

In [ ]:
#some analysis
import pandas as pd
ht=pd.read_csv('annual_household_control_totals_pandas.csv')

In [ ]:
ht.groupby('year').total_number_of_households.sum()

In [ ]:
ht[(ht.large_area_id==115) & (ht.persons_min>=10)]